In [1]:
import numpy as np
import cv2
import time
import os
import concurrent.futures as multithreading
from scipy.stats import entropy

In [2]:
class Video:
    
    @staticmethod
    def compress(filename, output_dst=None, fraction=0.01):

        def get_indices(height, width, fraction):

            def to_2d(index):
                return (index // width, index % width)

            samples = np.random.permutation(width * height)[:int(width * height * fraction)]

            indices = [to_2d(index) for index in samples]

            return tuple(zip(*indices))


        def sample_frame(frame, fraction):
            height, width, nb_channels = frame.shape

            indices = get_indices(height, width, fraction)

            return np.expand_dims(frame[indices], axis=0)


        seed = np.array(np.random.randint(0, np.iinfo(np.int32).max))
        np.random.seed(seed)
        
        cap = cv2.VideoCapture(filename)
        framerate = np.array(cap.get(cv2.cv2.CAP_PROP_FPS))

        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        start_time = time.time()

        ret, frame = cap.read()
        dimensions = np.array(frame.shape)

        video = sample_frame(frame, fraction)
        curr_frame = 1

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            sampled_frame = sample_frame(frame, fraction)

            video = np.vstack((video, sampled_frame))

            curr_frame += 1
            print('Compressing Video: ' + str(int(100 * curr_frame / num_frames)) + 
                  '%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds', end='\r')

        cap.release()

        print('Compressing Video: 100%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds')

        if output_dst is None:
            output_dst = '../Results/CompressedVideos/' + filename.split('/')[-1].split('.')[0]
        
        np.savez_compressed(
            output_dst, 
            video=video, 
            dimensions=dimensions, 
            framerate=framerate,
            seed=seed,
            fraction=np.array(fraction)
        )
    
    @staticmethod
    def psnr(original, reconstructed, color_channels='bgr'):
        cap_original = cv2.VideoCapture(original)
        cap_reconstructed = cv2.VideoCapture(reconstructed)
        
        num_frames_original = int(cap_original.get(cv2.CAP_PROP_FRAME_COUNT))
        num_frames_reconstructed = int(cap_reconstructed.get(cv2.CAP_PROP_FRAME_COUNT))
        num_frames = min(num_frames_original, num_frames_reconstructed)
        
        mse_list = []
        max_pixel_value = np.iinfo(np.uint8).max
        
        start_time = time.time()
        curr_frame = 0
        
        while cap_original.isOpened() and cap_reconstructed.isOpened():
            ret_original, frame_original = cap_original.read()
            ret_reconstructed, frame_reconstructed = cap_reconstructed.read()
            
            if (not ret_original) or (not ret_reconstructed):
                break
            
            height = frame_original.shape[0]
            width = frame_original.shape[1]
            frame_mse = 0
            
            if color_channels == 'bgr':
                nb_channels = frame_original.shape[2]
                total_squared_error = np.sum(np.square(frame_original.astype(int) - 
                                                       frame_reconstructed.astype(int)))
                frame_mse = total_squared_error / (width * height * nb_channels)
            
            elif color_channels == 'ycrcb':
                frame_original = cv2.cvtColor(frame_original, cv2.COLOR_BGR2YCrCb)
                frame_reconstructed = cv2.cvtColor(frame_reconstructed, cv2.COLOR_BGR2YCrCb)
                total_squared_error = np.sum(np.square(frame_original[:,:,0].astype(int) - 
                                                       frame_reconstructed[:,:,0].astype(int)))
                frame_mse = total_squared_error / (width * height)
            
            mse_list.append(frame_mse)
            
            curr_frame += 1
            print('Computing PSNR: ' + str(int(100 * curr_frame / num_frames)) +
                  '%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds', end='\r')
        
        mse = np.mean(mse_list)
        psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
        
        print('Computing PSNR: 100%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds')
        
        cap_original.release()
        cap_reconstructed.release()
        
        return psnr


In [3]:
class Kernels:
    
    @staticmethod
    def kernel_2d(kernel_size, sigma):
        center = kernel_size // 2
        kernel = np.fromfunction(
            lambda x, y: 
                np.exp( -0.5 * ((x - center) ** 2 + (y - center) ** 2) / (sigma ** 2)), 
            (kernel_size, kernel_size), 
            dtype=float)

        return kernel
    
    @staticmethod
    def kernel_3d(kernel_size, sigma, num_kernels, sigma_time):
        center = kernel_size // 2
        kernels = np.fromfunction(
            lambda t, x, y: 
                np.exp( -0.5 * (
                                 (((x - center) ** 2 + (y - center) ** 2) / (sigma ** 2)) + 
                                  ((t ** 2) / (sigma_time ** 2))
                               )
                      ), 
            (num_kernels, kernel_size, kernel_size), 
            dtype=float
        )
        return kernels


In [232]:
class CompressedVideo:
    
    def __init__(self, filename):
        
        file_dict = np.load(filename)
        
        self.filename = filename.split('.npz')[0]
        self.height = file_dict['dimensions'][0]
        self.width = file_dict['dimensions'][1]
        self.nb_channels = file_dict['dimensions'][2]
        self.video = file_dict['video']
        self.num_frames = self.video.shape[0]
        self.framerate = file_dict['framerate']
        self.seed = file_dict['seed']
        self.fraction = file_dict['fraction']
        
        # Create indices from seed
        def indices_generator():
            np.random.seed(self.seed)
            for idx in range(self.num_frames):
                samples = np.random.permutation(self.width * self.height)[:int(self.width * self.height * self.fraction)]
                yield tuple(zip(*[(index // self.width, index % self.width) for index in samples]))
        
        self.indices = np.array([i for i in indices_generator()])
        
        
        self.defaults = {}
        self.defaults['sigma']           = np.sqrt((self.height * self.width) / 
                                                   (self.video[0].shape[0] * np.pi))
        self.defaults['kernel_size']     = 2 * (int(0.5 + 3 * self.defaults['sigma']) + 2) + 1
        self.defaults['num_time_frames'] = 15
        self.defaults['output_dst']      = self.filename + '_reconstructed.mov'
        self.defaults['multithreaded']   = True
        self.defaults['alpha']           = 0.95
        self.defaults['beta']            = 80
        self.defaults['window_width']    = 150
        self.defaults['window_overlap']  = 0.5
    
    
    def reconstruct(self, algorithm='efan2d', color_channels='bgr', verbose=False, **kwargs):
        
        if 'kernel_size' in kwargs:
            kernel_size = kwargs['kernel_size']
        else:
            kernel_size = self.defaults['kernel_size']
        
        if 'sigma' in kwargs:
            sigma = kwargs['sigma']
        else:
            sigma = self.defaults['sigma']
        
        if 'num_time_frames' in kwargs:
            num_time_frames = kwargs['num_time_frames']
        else:
            num_time_frames = self.defaults['num_time_frames']
        
        if 'output_dst' in kwargs:
            output_dst = kwargs['output_dst']
        else:
            output_dst = self.defaults['output_dst']
        
        if 'multithreaded' in kwargs:
            multithreaded = kwargs['multithreaded']
        else:
            multithreaded = self.defaults['multithreaded']
        
        if 'alpha' in kwargs:
            alpha = kwargs['alpha']
        else:
            alpha = self.defaults['alpha']
        
        if 'beta' in kwargs:
            beta = kwargs['beta']
        else:
            beta = self.defaults['beta']
        
        if 'window_width' in kwargs:
            window_width = kwargs['window_width']
        else:
            window_width = self.defaults['window_width']
        
        if 'window_overlap' in kwargs:
            window_overlap = kwargs['window_overlap']
        else:
            window_overlap = self.defaults['window_overlap']
        
        
        def display_progress(frame_index, start_time, filter_times, complete=False):
            if verbose:
                print('Reconstructing Video: ' + str(int(100 * frame_index / self.num_frames)) + '% ' + 
                      '\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds ' +
                      '\tFilter Time: '  + str(int(np.floor(np.sum(filter_times)))) + ' seconds', end='\r')

                if complete:
                    print('')
        
        
        def prepare_frame(frame_index):
            frame = self.video[frame_index]
            indices = self.indices[frame_index]

            black_frame = np.zeros((self.height, self.width, self.nb_channels))
                
            if color_channels == 'ycrcb':
                frame = cv2.cvtColor(np.expand_dims(frame, axis=0), cv2.COLOR_BGR2YCrCb)[0]
            
            black_frame[tuple(indices)] = frame

            new_channel = np.zeros((self.height, self.width, 1)) + 1e-10 # To avoid division by zero
            new_channel[tuple(indices)] = 1.0

            augmented_frame = np.append(black_frame, new_channel, axis=2)
            
            return augmented_frame
        
        
        def normalize_frame(frame):
            for i in range(self.nb_channels):
                frame[:,:,i] /= frame[:,:,-1]

            reconstructed_frame = (frame[:,:,:-1] + 0.5).astype(np.uint8)
            
            return reconstructed_frame
        
        
        def efan2d(output):
            """
            EFAN2D Algorithm
            """
            
            filter_times = []

            def filter_frame(frame_index, kernel):
                augmented_frame = prepare_frame(frame_index)

                st = time.time()
                filtered = cv2.filter2D(augmented_frame, -1, kernel, 
                                        borderType=cv2.BORDER_CONSTANT)
                filter_times.append(time.time() - st)
                
                return filtered
            
            
            def reconstruct_frame(frame_index, kernel):
                filtered = filter_frame(frame_index, kernel)
                
                reconstructed_frame = normalize_frame(filtered)
                # Not sure about this, paper formula for reconstruction does not include it
                #reconstructed_frame[tuple(indices)] = frame
                
                return reconstructed_frame
            
            
            start_time = time.time()
            
            kernel = Kernels.kernel_2d(kernel_size, sigma)
            
            display_progress(0, start_time, filter_times)
            
            for frame_index in range(self.num_frames):
                reconstructed_frame = reconstruct_frame(frame_index, kernel)
                
                if color_channels == 'ycrcb':
                    reconstructed_frame = cv2.cvtColor(reconstructed_frame, cv2.COLOR_YCrCb2BGR)
                
                output.write(reconstructed_frame)
                display_progress(frame_index, start_time, filter_times)
            
            display_progress(self.num_frames, start_time, filter_times, complete=True)


        def efan3d(output):
            """
            EFAN3D Algorithm
            """
            
            num_kernels = (num_time_frames + 1) // 2
            sigma_time = (num_kernels - 1) / 6
            
            filter_times = []
            
            def filter_frame(frame_index, kernels):
                augmented_frame = prepare_frame(frame_index)

                st = time.time()
                
                if multithreaded:
                    filtered_frame = [None for i in range(num_kernels)]

                    def apply_kernel(kernel_index):
                        filtered = cv2.filter2D(augmented_frame, -1, kernels[kernel_index], 
                                                borderType=cv2.BORDER_CONSTANT)
                        filtered_frame[kernel_index] = filtered

                    with multithreading.ThreadPoolExecutor(max_workers=num_kernels) as executor:
                        executor.map(apply_kernel, range(num_kernels))
                
                else:
                    filtered_frame = []
                    for kernel in kernels:
                        filtered = cv2.filter2D(augmented_frame, -1, kernel, 
                                                borderType=cv2.BORDER_CONSTANT)
                        filtered_frame.append(filtered)
                
                filter_times.append(time.time() - st)
                
                return filtered_frame
            
            
            def reconstruct_frame(frame_index, filtered_frames):
                filtered_sum = np.zeros((self.height, self.width, self.nb_channels + 1))
            
                nb_past_frames = min(frame_index, num_kernels - 1)
                for idx in range(nb_past_frames):
                    filtered_sum += filtered_frames[idx][nb_past_frames - idx]

                nb_future_frames = min(self.num_frames - frame_index, num_kernels)
                for idx in range(nb_future_frames):
                    filtered_sum += filtered_frames[min(frame_index, num_kernels - 1) + idx][idx]
                
                reconstructed_frame = normalize_frame(filtered_sum)
                
                return reconstructed_frame
            
            
            start_time = time.time()
            
            kernels = Kernels.kernel_3d(kernel_size, sigma, num_kernels, sigma_time)
            
            filtered_frames = []
            
            display_progress(0, start_time, filter_times)
            
            for frame_index in range(self.num_frames):
                
                if frame_index == 0:
                    for idx in range(num_kernels):
                        filtered_frames.append(filter_frame(idx, kernels))
                        
                elif frame_index < num_kernels:
                    filtered_frames.append(filter_frame((num_kernels - 1) + frame_index, kernels))
                    
                elif frame_index <= self.num_frames - num_kernels:
                    filtered_frames.append(filter_frame((num_kernels - 1) + frame_index, kernels))
                    filtered_frames = filtered_frames[1:]
                    
                else:
                    filtered_frames = filtered_frames[1:]
                
                reconstructed_frame = reconstruct_frame(frame_index, filtered_frames)
                
                if color_channels == 'ycrcb':
                    reconstructed_frame = cv2.cvtColor(reconstructed_frame, cv2.COLOR_YCrCb2BGR)
                
                output.write(reconstructed_frame)
                
                display_progress(frame_index, start_time, filter_times)
            
            display_progress(self.num_frames, start_time, filter_times, complete=True)
        
        
        def adefan(output):
            
            num_kernels = (num_time_frames + 1) // 2
            sigma_time = (num_kernels - 1) / 6
            uniform = np.ones(np.iinfo(np.uint8).max + 1) / (np.iinfo(np.uint8).max + 1)
            
            filter_times = []
            
            def intervals(length):
                return [slice(x, min(x + window_width, length)) for x in 
                        range(0, length - int(window_width * window_overlap), 
                              int(window_width * (1 - window_overlap)))]
        
        
            def window_color_distributions(window):

                return np.array([alpha * np.histogram(window[:,channel], 
                                                      bins=range(np.iinfo(np.uint8).max + 2), 
                                                      density=True)[0] +
                                 (1 - alpha) * uniform
                                 for channel in range(self.nb_channels)]).flatten()
            
            
            def preprocess_frame(frame_index, windows):
                augmented_frame = prepare_frame(frame_index)
                
                frame_color_distributions = [
                    window_color_distributions(
                        augmented_frame[window][augmented_frame[window][:,:,-1] == 1][:,:-1]
                    ) for window in windows
                ]
                
                return augmented_frame, frame_color_distributions
            
            
            def compute_max_divergence(frame_index, frames_color_distributions):
                
                curr_frame_distributions = frames_color_distributions[min(frame_index, num_kernels - 1)]
                
            
            
            start_time = time.time()
            
            windows = [(x, y) for x in intervals(self.width) for y in intervals(self.height)]
            augmented_frames = []
            frames_color_distributions = []
            
            for frame_index in  range(self.num_frames):
                
                st = time.time()
                
                if frame_index == 0:
                    for idx in range(num_kernels):
                        augmented_frame, frame_color_distributions = preprocess_frame(idx, windows)
                        
                        augmented_frames.append(augmented_frame)
                        frames_color_distributions.append(frame_color_distributions)
                
                elif frame_index < num_kernels:
                    augmented_frame, frame_color_distributions = preprocess_frame(idx, windows)
                    
                    augmented_frames.append(augmented_frame)
                    frames_color_distributions.append(frame_color_distributions)
                    
                elif frame_index <= self.num_frames - num_kernels:
                    augmented_frame, frame_color_distributions = preprocess_frame(idx, windows)
                    
                    augmented_frames.append(augmented_frame)
                    frames_color_distributions.append(frame_color_distributions)
                    
                    augmented_frames           = augmented_frames[1:]
                    frames_color_distributions = frames_color_distributions[1:]
                    
                else:
                    augmented_frames           = augmented_frames[1:]
                    frames_color_distributions = frames_color_distributions[1:]
                
                
                
                
                
                
                
                filter_times.append(time.time() - st)
                display_progress(frame_index, start_time, filter_times)
            display_progress(self.num_frames, start_time, filter_times, complete=True)
                
            
            
            
            #############################################
        
        
        output = cv2.VideoWriter(
                    output_dst, 
                    cv2.VideoWriter_fourcc('M','J','P','G'), 
                    self.framerate, 
                    (self.width, self.height)
                 )
        
        if algorithm == 'efan2d':
            efan2d(output)
        elif algorithm == 'efan3d':
            efan3d(output)
        elif algorithm == 'adefan':
            adefan(output)
        
        output.release()
    

In [146]:
def prepare_frame(idx):
    frame = seal.video[idx]
    indices = seal.indices[idx]

    black_frame = np.zeros((seal.height, seal.width, seal.nb_channels))
    black_frame[tuple(indices)] = frame

    new_channel = np.zeros((seal.height, seal.width, 1)) + 1e-10 # To avoid division by zero
    new_channel[tuple(indices)] = 1.0

    augmented_frame = np.append(black_frame, new_channel, axis=2)

    return augmented_frame

In [147]:
af = prepare_frame(0)
af2 = prepare_frame(1)

In [218]:
(af[:150,:150])[(af[:150,:150])[:,:,-1] == 1][:,:-1]

array([[250., 254., 255.],
       [244., 248., 255.],
       [246., 250., 255.],
       [249., 253., 255.],
       [249., 253., 255.],
       [248., 252., 255.],
       [249., 253., 255.],
       [248., 252., 255.],
       [239., 243., 252.],
       [243., 247., 255.],
       [241., 245., 254.],
       [244., 248., 255.],
       [249., 253., 255.],
       [243., 247., 255.],
       [242., 246., 255.],
       [239., 243., 252.],
       [243., 247., 255.],
       [238., 242., 251.],
       [245., 249., 255.],
       [239., 243., 252.],
       [250., 254., 255.],
       [248., 252., 255.],
       [246., 250., 255.],
       [237., 241., 250.],
       [241., 245., 254.],
       [244., 248., 255.],
       [232., 236., 245.],
       [248., 252., 255.],
       [243., 248., 255.],
       [243., 248., 255.],
       [248., 252., 255.],
       [241., 248., 255.],
       [241., 248., 255.],
       [238., 242., 251.],
       [242., 246., 255.],
       [242., 247., 255.],
       [244., 248., 255.],
 

In [149]:
window_width = 150
window_overlap = 0.5

def intervals(length):
    return [slice(x, min(x + window_width, length)) for x in 
            range(0, length - int(window_width * window_overlap), 
                  int(window_width * (1 - window_overlap)))]

windows = [(x, y) for x in intervals(seal.width) for y in intervals(seal.height)]
len(windows)

32

In [150]:
st = time.time()
for window in windows:
    af[window][af[window][:,:,3] == 1][:,:-1]
print(time.time() - st)

0.004102945327758789


In [183]:
alpha = 0.95
uniform = np.ones(np.iinfo(np.uint8).max + 1) / (np.iinfo(np.uint8).max + 1)
def window_color_distributions(window):

    return np.array([alpha * np.histogram(window[:,channel], 
                                          bins=range(np.iinfo(np.uint8).max + 2), 
                                          density=True)[0] +
                     (1 - alpha) * uniform
                     for channel in range(seal.nb_channels)]).flatten()

In [204]:
d1 = window_color_distributions(af[windows[1]][af[windows[1]][:,:,-1] == 1][:,:-1])
d2 = window_color_distributions(af2[windows[1]][af2[windows[1]][:,:,-1] == 1][:,:-1])

In [205]:
entropy(d1,d2)

0.4709375744677732

In [5]:
class Validation:
    
    resources_folder = '../Resources/Videos/'
    compressed_folder = '../Results/CompressedVideos/'
    reconstructed_folder = '../Results/ReconstructedVideos/'
    
    @staticmethod
    def compress_videos(motion='both'):
        
        def compress_folder_content(folder):
            suffix = folder + '/'
            in_folder = Validation.resources_folder + suffix
            out_folder = Validation.compressed_folder + suffix
            
            for video in os.listdir(in_folder):
                video_name = video.split('.')[0]
                Video.compress(in_folder + video, out_folder + video_name)
        
        if motion == 'absent' or motion == 'both':
            compress_folder_content('MotionAbsent')
        
        if motion == 'present' or motion == 'both':
            compress_folder_content('MotionPresent')
    
    @staticmethod
    def reconstruct_videos(algorithm='efan2d', motion='both', extension='.mov', color_channels='bgr'):
        
        def reconstruct_folder_content(folder):
            suffix = folder + '/'
            algorithm_folder = algorithm.upper() + '/'
            in_folder = Validation.compressed_folder + suffix
            out_folder = Validation.reconstructed_folder + algorithm_folder + suffix
            
            for video in os.listdir(in_folder):
                video_name = video.split('.')[0]
                out_filename = video_name + extension
                compressed_video = CompressedVideo(in_folder + video)
                compressed_video.reconstruct(algorithm=algorithm, 
                                             output_dst=out_folder+out_filename, 
                                             color_channels=color_channels)
        
        if motion == 'absent' or motion == 'both':
            reconstruct_folder_content('MotionAbsent')

        if motion == 'present' or motion == 'both':
            reconstruct_folder_content('MotionPresent')
    
    @staticmethod
    def compute_mean_psnr(algorithm='efan2d', motion='absent', color_channels='bgr'):
        
        def compute_folder_mean_psnr(folder):
            psnr_list = []
            
            suffix = folder + '/'
            original_folder = Validation.resources_folder + suffix
            algorithm_folder = algorithm.upper() + '/'
            reconstructed_folder = Validation.reconstructed_folder + algorithm_folder + suffix
            
            for original, reconstructed in zip(os.listdir(original_folder), os.listdir(reconstructed_folder)):
                psnr = Video.psnr(original_folder + original, 
                                  reconstructed_folder + reconstructed, 
                                  color_channels=color_channels)
                psnr_list.append(psnr)
            
            return np.mean(psnr_list)
        
        if motion == 'absent':
            return compute_folder_mean_psnr('MotionAbsent')
        
        if motion == 'present':
            return compute_folder_mean_psnr('MotionPresent')


In [6]:
Video.compress('Resources/VideoSamples/Seal.mp4')

Compressing Video: 100%	Time Elapsed: 1 seconds


In [233]:
seal = CompressedVideo('../Results/CompressedVideos/Seal.npz')

In [52]:
seal.reconstruct(algorithm='efan2d', color_channels='ycrcb', output_dst='../Results/seal_ef2.mov', verbose=True)

Reconstructing Video: 100% 	Time Elapsed: 8 seconds 	Filter Time: 4 seconds


In [9]:
seal.reconstruct(algorithm='efan3d', multithreaded=True, output_dst='../Results/seal_ef3.mov', verbose=True)

In [10]:
seal.reconstruct(algorithm='efan3d', multithreaded=False, output_dst='../Results/seal_ef3singlethreaded.mov', verbose=True)

Using smaller resolutions seems much more reasonnable given the computational costs of hd videos.
Using multithreading for EFAN3D leads to a speedup of a factor 2 in runtime.

In [235]:
#seal.reconstruct(algorithm='adefan', output_dst='../Results/seal_adefan.mov', verbose=True)

In [117]:
Video.psnr('Resources/VideoSamples/Seal.mp4', '../Results/seal_ef2.mov', color_channels='ycrcb')

Computing PSNR: 100%	Time Elapsed: 0 seconds


21.823762138725613

In [112]:
Validation.compress_videos()

Compressing Video: 100%	Time Elapsed: 4 seconds
Compressing Video: 100%	Time Elapsed: 0 seconds
Compressing Video: 100%	Time Elapsed: 1 seconds
Compressing Video: 100%	Time Elapsed: 4 seconds
Compressing Video: 100%	Time Elapsed: 1 seconds
Compressing Video: 100%	Time Elapsed: 0 seconds
Compressing Video: 100%	Time Elapsed: 1 seconds
Compressing Video: 100%	Time Elapsed: 0 seconds
Compressing Video: 100%	Time Elapsed: 1 seconds
Compressing Video: 100%	Time Elapsed: 3 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 3 seconds
Compressing Video: 100%	Time Elapsed: 3 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 1 

In [113]:
Validation.reconstruct_videos(algorithm='efan2d', color_channels='ycrcb')

Reconstructing Video: 100% 	Time Elapsed: 27 seconds 	Filter Time: 15 seconds
Reconstructing Video: 100% 	Time Elapsed: 3 seconds 	Filter Time: 2 seconds
Reconstructing Video: 100% 	Time Elapsed: 8 seconds 	Filter Time: 4 seconds
Reconstructing Video: 100% 	Time Elapsed: 26 seconds 	Filter Time: 14 seconds
Reconstructing Video: 100% 	Time Elapsed: 11 seconds 	Filter Time: 6 seconds
Reconstructing Video: 100% 	Time Elapsed: 5 seconds 	Filter Time: 2 seconds
Reconstructing Video: 100% 	Time Elapsed: 13 seconds 	Filter Time: 7 seconds
Reconstructing Video: 100% 	Time Elapsed: 6 seconds 	Filter Time: 3 seconds
Reconstructing Video: 100% 	Time Elapsed: 12 seconds 	Filter Time: 7 seconds
Reconstructing Video: 100% 	Time Elapsed: 23 seconds 	Filter Time: 13 seconds
Reconstructing Video: 100% 	Time Elapsed: 20 seconds 	Filter Time: 11 seconds
Reconstructing Video: 100% 	Time Elapsed: 13 seconds 	Filter Time: 7 seconds
Reconstructing Video: 100% 	Time Elapsed: 18 seconds 	Filter Time: 10 second

In [106]:
Validation.compute_mean_psnr(motion='absent')

Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds


24.222866647707434

In [107]:
Validation.compute_mean_psnr(motion='present')

Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing 

24.575506557571156

In [114]:
Validation.compute_mean_psnr(motion='absent', color_channels='ycrcb')

Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds


24.43480608669383

In [115]:
Validation.compute_mean_psnr(motion='present', color_channels='ycrcb')

Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing 

25.00315016637109

In [29]:
##################################
##### Optimization Benchmark #####
##################################

height = 720
width = 1280
num_channels = 3
length = 38 #seconds
framerate = 24
num_frames = length * framerate
anchor_x = 400
anchor_y = 400
num_rand_pixels = (height * width) // 100

kernel_size = 71
kernel = Kernels.kernel_2d(kernel_size, 20)
r = (kernel_size - 1) // 2

###########################
##### Naive Operation #####
###########################

st = time.time()

kernel_dict = {}
for i in range(256):
    kernel_dict[i] = kernel * i

filtered = np.zeros((height, width, num_channels + 1));

for idx in range(num_rand_pixels // 100):
    k = kernel_dict[np.random.randint(0,256)]
    for c in range(num_channels + 1):
        for y in range(kernel_size):
            for x in range(kernel_size):
                filtered[anchor_x-r+y,anchor_y-r+x,c] += k[x,y]
tot_time_naive = (time.time() - st) * num_frames * 100
print('Naive python operations / array indexing:', np.around(tot_time_naive / 3600, 1), 'hours')

###########################
##### Numpy Optimized #####
###########################

st = time.time()

filtered = filtered = np.zeros((height, width, num_channels + 1));

kernel_dict = {}
for i in range(256):
    kernel_dict[i] = kernel * i

for i in range(num_rand_pixels):
    for c in range(num_channels + 1):
        filtered[anchor_y-r:anchor_y+r+1, anchor_x-r:anchor_x+r+1, c] += kernel_dict[np.random.randint(0,256)]

tot_time_numpy = (time.time() - st) * num_frames
print('Numpy (C) operations / array indexing:', np.around(tot_time_numpy / 60, 1), 'minutes')
print('Numpy speedup:', np.around(tot_time_naive / tot_time_numpy, 1), 'times faster')

Naive python operations / array indexing: 29.6 hours
Numpy (C) operations / array indexing: 9.6 minutes
Numpy speedup: 185.0 times faster


The equivalent of the MATLAB / C++ implementation would take over 29 hours to reconstruct the video using python operations and almost 10 minutes using numpy operations. Since numpy is already written in C there isn't much more to do to speed up the additions / multiplications / array indexing. Even using numpy this is still more than 4 times slower than using opencv to interpolate the value of each pixel (Probably because of the time gained by using filter2D which uses the FFT algorithm).

In [22]:
######################################
##### MATLAB / Python comparison #####
######################################

import scipy.io as sio

frame = cv2.imread('Resources/ImageComparison/lena.png')
width = frame.shape[1]
height = frame.shape[0]
nb_channels = frame.shape[2]

randvec = sio.loadmat('Resources/ImageComparison/randvec.mat')['randind'][0] - 1 # MATLAB indexing starts at 1

st = time.time() # Start counting time

indices = [(index % height, index // height) for index in randvec] # MATLAB is column major so we convert to row major
indices = tuple(zip(*indices))

black_frame = np.zeros((height, width, nb_channels))
black_frame[indices] = frame[indices]

new_channel = np.zeros((height, width, 1)) + 1e-10 # To avoid division by zero
new_channel[indices] = 1.0

augmented_frame = np.append(black_frame, new_channel, axis=2)

sigma = np.sqrt((height * width) / (len(indices[0]) * np.pi))
kernel_size = 2 * (int(0.5 + 3 * sigma) + 2) + 1

kernel = Kernels.kernel_2d(kernel_size, sigma)

filtered = cv2.filter2D(augmented_frame, -1, kernel, borderType=cv2.BORDER_CONSTANT)
for i in range(nb_channels):
    filtered[:,:,i] /= filtered[:,:,-1]
reconstructed_frame = (filtered[:,:,:-1] + 0.5).astype(np.uint8)

print('Total time elapsed', time.time() - st, 'seconds')

cv2.imwrite('Resources/ImageComparison/lena_python.png', reconstructed_frame);

Total time elapsed 0.05006265640258789 seconds


Our method takes the same amount of time as the MATLAB / C++ implementation to reconstruct the image

In [23]:
lena_matlab = cv2.imread('Resources/ImageComparison/lena_matlab.png')
lena_python = cv2.imread('Resources/ImageComparison/lena_python.png')

print('Number of different channel values:', np.count_nonzero(lena_matlab - lena_python))

Number of different channel values: 2


Only 6 channel values differ between the two images so the methods are equivalent (those values are probably because of different rounding / precision between python and C++)

In [24]:
lena_original = cv2.imread('Resources/ImageComparison/lena.png')
lena_matlab = cv2.imread('Resources/ImageComparison/lena_matlab.png')
lena_python = cv2.imread('Resources/ImageComparison/lena_python.png')

width, height, nb_channels = lena_original.shape
max_pixel_value = np.iinfo(np.uint8).max

matlab_mse = np.sum(np.square(lena_original - lena_matlab)) / (width * height * nb_channels)
python_mse = np.sum(np.square(lena_original - lena_python)) / (width * height * nb_channels)
matlab_psnr = 10 * np.log10((max_pixel_value ** 2) / matlab_mse)
python_psnr = 10 * np.log10((max_pixel_value ** 2) / python_mse)

print('MATLAB MSE:', matlab_mse, '\nPython MSE:', python_mse)
print('MATLAB PSNR:', matlab_psnr, '\nPython PSNR:', python_psnr)

MATLAB MSE: 61.019219716389976 
Python MSE: 61.01919428507487
MATLAB PSNR: 30.276137110691067 
Python PSNR: 30.276138920724392


# ONLY CODE TESTS BELOW THIS CELL

In [63]:
alpha = 0.95
uniform = np.ones(256) / 256

st = time.time()
for i in range(seal.video.shape[0]):
    a = np.histogram(seal.video[0][:,0], 
                     bins=range(np.iinfo(np.uint8).max + 2), density=True)[0]

    b = np.histogram(seal.video[1][:,0], 
                     bins=range(np.iinfo(np.uint8).max + 2), density=True)[0]

    a = alpha * a + (1 - alpha) * uniform
    b = alpha * b + (1 - alpha) * uniform
    
    for i in range(200):
        entropy(a, b)
print(time.time() - st)

0.9000978469848633
